# Data Scraping

In this notebook, we will fetch and download all the necessary datasets and store them in their associated files within the "data" file

### Importing Libraries and Functions

In [1]:
import sys
sys.path.append('../')
import os
from urllib.request import urlretrieve
import pandas as pd
from json import dump
from scripts.scrape_zip import get_zip
from scripts.scrape_xlsx import get_xlsx
from scripts.parallelised_scrape import generate_url_list, fetch_all_rental_data
from scripts.scrape_population import fetch_yearly_pop_projection

### Defining Global Variables - changes pending

In [2]:
# CONSTANTS
BASE_URL = "https://www.domain.com.au"
LANDING_PATH = "data/landing"
RAW_PATH = "data/raw"

### Fetching All The '*domain.com*' Rental Property Links

In [ ]:
# Generating all links
url_links = generate_url_list(BASE_URL)

In [4]:
# Checking we have the correct number of properties
print(len(url_links))

13823


### Fetching The Data From Each Of The URLs 

In [5]:
# Scraping all data
rental_data = fetch_all_rental_data(url_links)

  6%|▌         | 794/13823 [01:56<26:10,  8.30it/s]  

Issue with https://www.domain.com.au/3-jean-street-lalor-vic-3075-17181912: 'NoneType' object has no attribute 'find'


 23%|██▎       | 3177/13823 [08:16<30:00,  5.91it/s]

Issue with https://www.domain.com.au/22-200-wattletree-road-malvern-vic-3144-17185487: 'NoneType' object has no attribute 'text'


 37%|███▋      | 5145/13823 [12:54<15:56,  9.07it/s]  

...

 72%|███████▏  | 9979/13823 [24:32<07:33,  8.47it/s]

Issue with https://www.domain.com.au/3-devon-court-mount-martha-vic-3934-17168700: 'NoneType' object has no attribute 'text'


 81%|████████▏ | 11248/13823 [27:48<06:22,  6.74it/s]

Issue with https://www.domain.com.au/21-nyah-st-keilor-east-vic-3033-17078424: 'NoneType' object has no attribute 'find'


100%|██████████| 13823/13823 [33:22<00:00,  6.90it/s]


In [6]:
# Creates the directory for the rental data if it doesn't exist
if not os.path.exists(f"../{LANDING_PATH}"):
    os.makedirs(f"../{LANDING_PATH}")

# Save the data in the data/landing directory
with open(f'../{LANDING_PATH}/all_properties_metadata.json', 'w') as f:
    dump(rental_data, f)
    print(f"File saved in the {LANDING_PATH} directory")

File saved in the data/landing directory


## Scraping Suburb Info

In [8]:
# Get post codes 
from scripts.scrape_oldlistings import scrape_postcodes_from_file

In [9]:
postcode_df = scrape_postcodes_from_file()
postcode_df.head()

,suburb,postcode
0,Abbotsford,3067
1,Aberfeldie,3040
2,Aberfeldy,3825
3,Acheron,3714
4,Addington,3352


### Scraping Additional Data

* SA2 Districts in Victoria shapefile

In [2]:
get_zip(url="https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip", output_dir = '../data/SA2/SA2')

* Historical Population by SA2 district

In [3]:
get_zip(url='https://www.abs.gov.au/statistics/people/population/regional-population/2022-23/32180_ERP_2023_SA2_GDA2020.zip',output_dir='../data/population/population')

* Victorian Homelessness data

In [5]:
get_xlsx(url='https://www.abs.gov.au/statistics/people/housing/estimating-homelessness-census/2021/20490do005_2021.xlsx', output_dir='../data/homlessness/homelessness')

Downloaded file saved to: ../data/homlessness/homelessness.xlsx


'../data/homlessness/homelessness.xlsx'

* Socio-economic Indexes per SA2 district

In [6]:
get_xlsx(url='https://www.abs.gov.au/statistics/people/people-and-communities/socio-economic-indexes-areas-seifa-australia/2021/Statistical%20Area%20Level%202%2C%20Indexes%2C%20SEIFA%202021.xlsx',output_dir='../data/socioeconomic/socioeconomic')

Downloaded file saved to: ../data/socioeconomic/socioeconomic.xlsx


'../data/socioeconomic/socioeconomic.xlsx'

* 5-Yearly Projected Population size per SA2 district

In [5]:
# 5 year projections links
url = "https://www.planning.vic.gov.au/__data/assets/excel_doc/0036/691659/VIF2023_SA2_Pop_Age_Sex_Projections_to_2036_Release_2.xlsx"

# Retrieving and saving the population projection data
pop_data = urlretrieve(url, f"../{LANDING_PATH}/5yearpopproj_perSA2.xlsx")

pop_data.head()

* Yearly Projected Population size per SA2 district

In [4]:
# Yearly projections links
url = "https://www.gen-agedcaredata.gov.au/getmedia/564291d5-8e25-4b2e-90e6-171f554cfce9/Victoria.csv"

headers = {"User-Agent":
           "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"}


# Retrieving and saving the population projection data
pop_data = fetch_yearly_pop_projection(url, f"../{LANDING_PATH}", f"../{LANDING_PATH}/yearly_pop_projection_perSA.csv", headers)


Data successfully written to ../data/landing/yearly_pop_projection_perSA.csv
<class 'bytes'>
